In [ ]:
import sqlite3
import os
import pandas as pd

# change to the directory where the files are stored
os.chdir("/Users/christopherpang/Desktop/Main/SIM uni year 2/Programming for data science/Coursework/dataverse_files")

try:
    os.remove('airline2.db')
except OSError:
    pass



# ======= create database =======
conn = sqlite3.connect('airline2.db')

data1999= pd.read_csv("1999.csv")
print(data1999.shape)
data2000= pd.read_csv("2000.csv")
print(data2000.shape)
data1999.info(verbose =False)
data2000.info(verbose = False)

#df2 = pd.concat([df, df1], ignore_index=True, sort=False)
ontime_df = pd.concat([data1999,data2000], ignore_index=True, sort=False)
airports_df = pd.read_csv("airports.csv")
carriers_df = pd.read_csv("carriers.csv")
planes_df = pd.read_csv("plane-data.csv")
#print(ontime_df.head)


# ======= statistics summary =======
summaryAirports = airports_df.describe(include='all')
#print(summaryAirports)

summaryCarriers = carriers_df.describe(include='all')
#print(summaryCarriers)

summaryPlanes = planes_df.describe(include='all')
#print(summaryPlanes)

summaryOntime = ontime_df.describe(include='all')
#print(summaryOntime)

c = conn.cursor()
c.execute('''
CREATE TABLE ontime (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')
conn.commit()



for year in range(1999, 2000):
    ontime = pd.read_csv(str(year)+".csv")
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)

conn.commit()



timeOfday= c.execute(''' SELECT DepTime AS DepTime FROM ontime 
                         WHERE ArrDelay < '5' AND DepDelay < '5' 
    
''').fetchall()
print(pd.DataFrame(timeOfday))
      


(5527884, 29)
(5683047, 29)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5527884 entries, 0 to 5527883
Columns: 29 entries, Year to LateAircraftDelay
dtypes: float64(13), int64(12), object(4)
memory usage: 1.2+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5683047 entries, 0 to 5683046
Columns: 29 entries, Year to LateAircraftDelay
dtypes: float64(13), int64(12), object(4)
memory usage: 1.2+ GB
